In [2]:
import pyddm.plot
from pyddm import Model, set_N_cpus
from pyddm.models import DriftConstant, DriftLinear, NoiseConstant,Bound,Drift, BoundConstant, OverlayNonDecision, ICPointSourceCenter,BoundCollapsingLinear
from pyddm.functions import fit_adjust_model, display_model

from pyddm import Fittable, Fitted, Sample
from pyddm.models import LossRobustBIC, LossBIC, LossLikelihood, LossRobustLikelihood,LossSquaredError
from pyddm.functions import fit_adjust_model

import copy

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import scipy
from statannot import add_stat_annotation
import glob

from sbi import utils

In [ ]:
def transform_df_DDM2v2a2c2ter(df):
    SAT = [copy.copy(e) for _ in range(len(df)) for e in ["Speed"]] + [copy.copy(e) for _ in range(len(df)) for e in ["Accuracy"]]
    df_speed = df.loc[:, ["Drift Rate Speed", "Boundary Separation Speed", "Non-decision Time Speed", "Collapse Speed"]]
    df_speed = df_speed.rename(columns={"Drift Rate Speed":"Drift Rate",
                                        "Boundary Separation Speed": "Boundary Separation",
                                        "Non-decision Time Speed": "Non-decision Time",
                                        "Collapse Speed":"Collapse Rate"})

    df_accuracy = df.loc[:, ["Drift Rate Accuracy", "Boundary Separation Accuracy", "Non-decision Time Accuracy", "Collapse Accuracy"]]
    df_accuracy = df_accuracy.rename(columns={"Drift Rate Accuracy": "Drift Rate", 
                                              "Boundary Separation Accuracy": "Boundary Separation",
                                              "Non-decision Time Accuracy": "Non-decision Time",
                                              "Collapse Accuracy":"Collapse Rate"})

    df_concat = pd.concat((df_speed, df_accuracy), ignore_index=True)
    df_concat["SAT"] = SAT

    df_melted = pd.melt(df_concat, id_vars = ["SAT"], 
                            value_vars=['Drift Rate', 'Boundary Separation','Non-decision Time', 'Collapse Rate'],
                            var_name = "Parameter", value_name = "Value")
    return df_melted

In [ ]:
def get_DDM_result_sub_2v2c2ter(samp, v = [0,6], a = [0.5,8], c = [0,4], ter=[0.1,1],
                                dx=.01, dt=.01, T_dur=5):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftConstantSpeedAcc(Vspeed=Fittable(minval=v[0], maxval=v[1]),
                                                  Vacc=Fittable(minval=v[0], maxval=v[1])), 
                      noise=NoiseConstant(noise=1),
                      bound=BoundCollapsingLinearSAT(B      =   Fittable(minval=a[0], maxval=a[1]),
                                                     cSpeed =   Fittable(minval=c[0], maxval=c[1]),
                                                     cAcc   =   Fittable(minval=c[0], maxval=c[1])),
                      
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),

                      ),
                      dx=dx, dt=dt, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit, fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,7),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'drift': 'Drift Rate Speed',
                            'drift': 'Drift Rate Accuracy',
                            'B': 'Boundary Separation',
                            'cSpeed': 'Collapse Speed',
                            'cAcc':   'Collapse Accuracy',
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [ ]:
def get_DDM_result_sub_2v2a2c2ter(samp, v = [0,6], a = [0.5,8], c = [0,4], ter=[0.1,1],
                                  dx=.01, dt=.01, T_dur=5):

    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftConstantSpeedAcc(Vspeed = Fittable(minval=v[0], maxval=v[1]),
                                                  Vacc = Fittable(minval=v[0], maxval=v[1])), 
                      noise=NoiseConstant(noise=1),
                      bound=BoundCollapsingLinearSAT(BSpeed=  Fittable(minval=a[0], maxval=a[1]),
                                                     BAcc =   Fittable(minval=a[0], maxval=a[1]),
                                                     cSpeed = Fittable(minval=c[0], maxval=c[1]),
                                                     cAcc =   Fittable(minval=c[0], maxval=c[1])),
                      
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),

                      ),
                      dx=dx, dt=dt, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,8),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'Vspeed': 'Drift Rate Speed',
                            'Vacc': 'Drift Rate Accuracy',
                            'BSpeed': 'Boundary Separation Speed',
                            'BAcc':   'Boundary Separation Accuracy',
                            'cSpeed': 'Collapse Speed',
                            'cAcc':   'Collapse Accuracy',
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [ ]:
def get_DDM_result_sub_2a2ter(samp, v = [0,6], a = [0.5,8], ter=[0.1,1], T_dur=3):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftConstant(drift=Fittable(minval=v[0], maxval=v[1])), 
                      noise=NoiseConstant(noise=1),
                      bound=BoundSpeedAcc(
                          Bspeed=Fittable(minval=a[0], maxval=a[1]),
                          Bacc=Fittable(minval=a[0], maxval=a[1])),
                      
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),

                      ),
                      dx=.001, dt=.001, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,5),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'drift': 'Drift Rate',
                            'Bspeed': 'Boundary Separation Speed',
                            'Bacc': 'Boundary Separation Accuracy',
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [ ]:
def get_DDM_result_sub_2v2a2ter(samp, v = [0,6], a = [0.5,8], ter=[0.1,1], T_dur=5):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftConstantSpeedAcc(Vspeed = Fittable(minval=v[0], maxval=v[1]),
                                                  Vacc = Fittable(minval=v[0], maxval=v[1])), 
                      noise=NoiseConstant(noise=1),
                      bound=BoundSpeedAcc(
                          Bspeed=Fittable(minval=a[0], maxval=a[1]),
                          Bacc=Fittable(minval=a[0], maxval=a[1])),
                      
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),

                      ),
                      dx=.01, dt=.01, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,6),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'Vspeed': 'Drift Rate Speed',
                            'Vacc': 'Drift Rate Accuracy',
                            'Bspeed': 'Boundary Separation Speed',
                            'Bacc': 'Boundary Separation Accuracy',
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [ ]:
def get_OUM_result_sub_2a2ter(samp, v = [0,6], a = [0.5,8], k = [-10,10], ter=[0.1,1], T_dur=3):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftLinear(drift=Fittable(minval=v[0], maxval=v[1]),t = 0,
                                        x=Fittable(minval=k[0], maxval=k[1])),                       
                      noise=NoiseConstant(noise=1),
                      bound=BoundSpeedAcc(
                          Bspeed=Fittable(minval=a[0], maxval=a[1]),
                          Bacc=Fittable(minval=a[0], maxval=a[1])),
                      
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),

                      ),
                      dx=.001, dt=.001, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,6),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'drift': 'Drift Rate',
                            'Bspeed': 'Boundary Separation Speed',
                            'Bacc': 'Boundary Separation Accuracy',
                            'K': "Self-excitation",
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [ ]:
def get_OUM_result_sub_2k2ter(samp, v = [0,6], a = [0.5,8], k = [-10,10], ter=[0.1,1], T_dur=3):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftExcitationSpeedAcc(drift=Fittable(minval=v[0], maxval=v[1]),
                                                    Kspeed=Fittable(minval=k[0], maxval=k[1]),
                                                    Kacc=Fittable(minval=k[0], maxval=k[1])),                              
                      noise=NoiseConstant(noise=1),
                      bound=BoundConstant(B=Fittable(minval=a[0], maxval=a[1])),
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),
                      ),
                      dx=.001, dt=.001, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,6),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'drift': 'Drift Rate',
                            'B': 'Boundary Separation',
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy',
                            'Kspeed': "Self-excitation Speed",
                            'Kacc': "Self-excitation Accuracy",
                           })
    return df

In [ ]:
def get_OUM_result_sub_2v2a2ter(samp, v = [0,6], a = [0.5,8], k = [-10,10], ter=[0.1,1], T_dur=3):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftConstantSpeedAcc(Vspeed = Fittable(minval=v[0], maxval=v[1]),
                                                  Vacc = Fittable(minval=v[0], maxval=v[1]),
                                                  K = Fittable(minval=k[0], maxval=k[1])), 
                      noise=NoiseConstant(noise=1),
                      bound=BoundSpeedAcc(
                          Bspeed=Fittable(minval=a[0], maxval=a[1]),
                          Bacc=Fittable(minval=a[0], maxval=a[1])),
                      
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),

                      ),
                      dx=.001, dt=.001, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,7),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'Vspeed': 'Drift Rate Speed',
                            'Vacc': 'Drift Rate Accuracy',
                            'Bspeed': 'Boundary Separation Speed',
                            'Bacc': 'Boundary Separation Accuracy',
                            'K': "Self-excitation",
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [ ]:
def get_OUM_result_sub_2v2k2ter(samp, v = [0,6], a = [0.5,8], k = [-10,10], ter=[0.1,1], 
                                dx=.01, dt=.01, T_dur=5):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftBothSpeedAcc(Vspeed = Fittable(minval=v[0], maxval=v[1]),
                                              Vacc = Fittable(minval=v[0], maxval=v[1]),
                                              Kspeed=Fittable(minval=k[0], maxval=k[1]),
                                              Kacc=Fittable(minval=k[0], maxval=k[1])), 
                      noise=NoiseConstant(noise=1),
                      bound=BoundConstant(B=Fittable(minval=a[0], maxval=a[1])),                      
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),

                      ),
                      dx=.01, dt=.01, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,7),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'Vspeed': 'Drift Rate Speed',
                            'Vacc': 'Drift Rate Accuracy',
                            'B': 'Boundary Separation',
                            'Kspeed': "Self-excitation Speed",
                            'Kacc': "Self-excitation Accuracy",
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [ ]:
def get_OUM_result_sub_full(samp, v = [0,6], a = [0.5,8], k = [-10,10], ter=[0.1,1]
                            ,dx=.01, dt=.01, T_dur=5):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftBothSpeedAcc(Vspeed = Fittable(minval=v[0], maxval=v[1]),
                                              Vacc = Fittable(minval=v[0], maxval=v[1]),
                                              Kspeed=Fittable(minval=k[0], maxval=k[1]),
                                              Kacc=Fittable(minval=k[0], maxval=k[1])), 
                      noise=NoiseConstant(noise=1),
                      bound=BoundSpeedAcc(
                          Bspeed=Fittable(minval=a[0], maxval=a[1]),
                          Bacc=Fittable(minval=a[0], maxval=a[1])),
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),

                      ),
                      dx=dx, dt=dt, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,8),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'Vspeed': 'Drift Rate Speed',
                            'Vacc': 'Drift Rate Accuracy',
                            'Bspeed': 'Boundary Separation Speed',
                            'Bacc': 'Boundary Separation Accuracy',
                            'Kspeed': "Self-excitation Speed",
                            'Kacc': "Self-excitation Accuracy",
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [ ]:
def get_DDM_result_sub(samp, SAT, v = [0,6], a = [0.5,8], ter=[0.1,1], T_dur=3):
    
    model_fit = Model(name='Simple model (fitted)',
                              drift=DriftConstant(drift=Fittable(minval=v[0], maxval=v[1])),
                              noise=NoiseConstant(noise=1),
                              bound=BoundConstant(B=Fittable(minval=a[0], maxval=a[1])),
                              overlay=OverlayNonDecision(nondectime=Fittable(minval=ter[0], maxval=ter[1])),
                              dx=.001, dt=.001, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,3),
                      columns = model_fit.get_model_parameter_names())
    df['Participant']=i+1
    df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'drift': 'Drift Rate',
                            'B': 'Boundary Separation',
                            'nondectime': "Non-decision Time"})

    
    df_all = pd.DataFrame(columns = ['Participant', 'drift', 'B', 'nondectime','SAT','BIC'])

    return df

In [ ]:
def get_OUM_result_sub_2a(samp, v = [0,6], a = [0.5,8], k = [-10,10], ter=[0.1,1], T_dur=3):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftLinear(drift=Fittable(minval=v[0], maxval=v[1]),
                                        t = 0, x=Fittable(minval=k[0], maxval=k[1])),                                noise=NoiseConstant(noise=1),
                      bound=BoundSpeedAcc(
                          Bspeed=Fittable(minval=a[0], maxval=a[1]),
                          Bacc=Fittable(minval=a[0], maxval=a[1])),
                      
                      overlay=OverlayNonDecision(nondectime=Fittable(minval=ter[0], maxval=ter[1])),
                      dx=.001, dt=.001, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,5),
                      columns = model_fit.get_model_parameter_names())
    
    df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'drift': 'Drift Rate',
                            'Bspeed': 'Boundary Separation Speed',
                            'Bacc': 'Boundary Separation Accuracy',

                            'nondectime': "Non-decision Time",
                            'x': "Self-excitation"})
    return df

In [ ]:
def get_OUM_result_sub_2k(samp, v = [0,6], a = [0.5,8], k = [-10,10], ter=[0.1,1], T_dur=3):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftSpeedAcc(drift = Fittable(minval=v[0], maxval=v[1]),
                                          Kspeed=Fittable(minval=k[0], maxval=k[1]),
                                          Kacc=Fittable(minval=k[0], maxval=k[1])),                      
                      noise=NoiseConstant(noise=1),
                      bound=BoundConstant(B=Fittable(minval=a[0], maxval=a[1])),
                      overlay=OverlayNonDecision(nondectime=Fittable(minval=ter[0], maxval=ter[1])),
                      dx=.001, dt=.001, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,5),
                      columns = model_fit.get_model_parameter_names())
    df['Participant']=i+1
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'drift': 'Drift Rate',
                            'B': 'Boundary Separation',
                            'nondectime': "Non-decision Time",
                            'Kspeed': 'Self-excitation Speed',
                            'Kacc': 'Self-excitation Accuracy'
                           })
    return df

In [ ]:
def get_OUM_result_sub(samp, SAT, v = [0,6], a = [0.5,8], k = [-10,10], ter=[0.1,1], T_dur=3):
    
    model_fit = Model(name='Simple model (fitted)',
                              drift=DriftLinear(drift=Fittable(minval=v[0], maxval=v[1]),
                                                t = 0, x=Fittable(minval=k[0], maxval=k[1])),                                noise=NoiseConstant(noise=1),
                              bound=BoundConstant(B=Fittable(minval=a[0], maxval=a[1])),
                              overlay=OverlayNonDecision(nondectime=Fittable(minval=ter[0], maxval=ter[1])),
                              dx=.001, dt=.001, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,4),
                      columns = model_fit.get_model_parameter_names())
    df['Participant']=i+1
    df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'drift': 'Drift Rate',
                            'B': 'Boundary Separation',
                            'nondectime': "Non-decision Time",
                            'x': "Self-excitation"})
    return df

In [3]:
def get_DDM_OUM_result(df_Dutilh2019, v, a, ter, k, T_dur=3):
    df_all = pd.DataFrame(columns = ['Participant', 'Drift Rate','Boundary Separation','Non-decision Time','Effective Inhibition','SAT'],)

    for i in range(len(df_Dutilh2019.index)):

        print(i)
        model = Model(name='Simple model (fitted)',
                      drift=DriftConstant(drift=df_Dutilh2019["Drift Rate"][i]),   
                      noise=NoiseConstant(noise=1),
                      bound=BoundConstant(B = df_Dutilh2019["Boundary Separation"][i]),
                      overlay=OverlayNonDecision(nondectime = df_Dutilh2019["Non-decision Time"][i]),
                      dx=.001, dt=.001, T_dur=3)

        sol = model.solve()
        samp = sol.resample(1000)


        model_fit = Model(name='Simple model (fitted)',
                      drift=DriftLinear(drift=Fittable(minval=v[0], maxval=v[1]),
                                        t = 0,
                                        x=Fittable(minval=k[0], maxval=k[1])),
                      noise=NoiseConstant(noise=1),
                      bound=BoundConstant(B=Fittable(minval=a[0], maxval=a[1])),
                      overlay=OverlayNonDecision(nondectime=Fittable(minval=ter[0], maxval=ter[1])),
                      dx=.001, dt=.001, T_dur=T_dur)

        fit_adjust_model(samp, model_fit,
                         fitting_method="differential_evolution",
                         lossfunction=LossRobustBIC, verbose=False)

        #df_sub = pd.DataFrame(columns = ['Frequency', 'SAT', 'Participant', 'v','a','ter','k'],)

        df_sub = pd.DataFrame({
                               'Participant': df_Dutilh2019["Participant"][i],
                               'Drift Rate': np.array(model_fit.parameters()['drift']['drift']), 
                               'Boundary Separation': np.array(model_fit.parameters()['bound']['B']),
                               'Non-decision Time': np.array(model_fit.parameters()['overlay']['nondectime']),
                               'Effective Inhibition': np.array(model_fit.parameters()['drift']['x']),
                               'SAT':df_Dutilh2019["SAT"][i]
                              }, index=[i])

        df_all = df_all.append(df_sub)

        print(df_sub)
    return(df_all)

In [ ]:
def sim_fit_plot_all(par, par_fitted, par_labels, par_fitted_labels, s = 4, figsize=(10,10)):
    
    param_num = par.shape[1]
    param_fitted_num = par_fitted.shape[1]
    
    fig, axes = plt.subplots(param_fitted_num, param_num, figsize=figsize)
    
    for i in range(param_num):
        for j in range(param_fitted_num):
            axes[j,i].scatter(par[:,i], par_fitted[:,j], s = s, facecolors='none', edgecolors='k')

    for m in range(param_fitted_num):
        axes[m,0].set_ylabel(par_fitted_labels[m],fontsize=18)#, rotation=0)
    
    for n in range(param_num):
        #axes[0,n].set_title(par_labels[n])
        axes[(param_fitted_num-1),n].set_xlabel(par_labels[n], fontsize=18)
    fig.align_ylabels(axes[:, :])
    fig.align_xlabels(axes[:, :])




In [ ]:
def standardize_result1(dataset, model):

    df100_DDM = pd.read_csv(model + "result" + dataset + ".csv")

    ## calculate mean and std of each param for one dataset
    df100_DDM["v_mean"] = [np.mean(df100_DDM["Drift Rate"])]*len(df100_DDM.index)
    df100_DDM["v_sd"] = [np.std(df100_DDM["Drift Rate"])]*len(df100_DDM.index)

    df100_DDM["a_mean"] = [np.mean(df100_DDM["Boundary Separation"])]*len(df100_DDM.index)
    df100_DDM["a_sd"] = [np.std(df100_DDM["Boundary Separation"])]*len(df100_DDM.index)

    df100_DDM["ter_mean"] = [np.mean(df100_DDM["Non-decision Time"])]*len(df100_DDM.index)
    df100_DDM["ter_sd"] = [np.std(df100_DDM["Non-decision Time"])]*len(df100_DDM.index)

    # standardize
    df100_DDM["Drift Rate"] = (df100_DDM["Drift Rate"] - df100_DDM["v_mean"])/df100_DDM["v_sd"]
    df100_DDM["Boundary Separation"] = (df100_DDM["Boundary Separation"] - df100_DDM["a_mean"])/df100_DDM["a_sd"]
    df100_DDM["Non-decision Time"] = (df100_DDM["Non-decision Time"] - df100_DDM["ter_mean"])/df100_DDM["ter_sd"]
    
    if "Effective Inhibition" in list(df100_DDM.columns):
        df100_DDM["k_mean"] = [np.mean(df100_DDM["Effective Inhibition"])]*len(df100_DDM.index)
        df100_DDM["k_sd"] = [np.std(df100_DDM["Effective Inhibition"])]*len(df100_DDM.index)
        df100_DDM["Effective Inhibition"] = (df100_DDM["Effective Inhibition"] - df100_DDM["k_mean"])/df100_DDM["k_sd"]

    
    return(df100_DDM)

In [9]:
def get_cohen_d(df_dif, model):

    dataset_model = np.array(df_dif[['Dataset','Model']].loc[0])
    mean = np.array(df_dif.groupby(['Dataset','Model'], as_index=False).agg("mean"))[0,2:]
    std = np.array(df_dif.groupby(['Dataset','Model'], as_index=False).agg("std"))[0,2:]
    d = mean/std
    arr = np.concatenate((np.expand_dims(dataset_model, axis = 1),np.expand_dims(d, axis = 1)))
    
    if model == "DDM":
        df = pd.DataFrame(arr.T, columns = ["Dataset", "Model", 'Drift Rate', 'Boundary Separation','Non-decision Time'])
        df_melted = pd.melt(df, id_vars = ["Dataset", "Model"],
                    value_vars=['Drift Rate', 'Boundary Separation','Non-decision Time'],
                    var_name = "Parameter", value_name = "Cohen's d")
    if model == "OUM" or model == "DDMOUM":
        df = pd.DataFrame(arr.T, columns = ["Dataset", "Model", 'Drift Rate', 'Boundary Separation','Non-decision Time',"Effective Inhibition"])
        df_melted = pd.melt(df, id_vars = ["Dataset", "Model"],
                    value_vars=['Drift Rate', 'Boundary Separation','Non-decision Time', "Effective Inhibition"],
                    var_name = "Parameter", value_name = "Cohen's d")
    return(df_melted)   

In [ ]:
def standardize_result(df100_DDM, model):

    ## calculate mean and std of each param for one dataset
    df100_DDM["v_mean"] = [np.mean(df100_DDM["Drift Rate"])]*len(df100_DDM.index)
    df100_DDM["v_sd"] = [np.std(df100_DDM["Drift Rate"])]*len(df100_DDM.index)

    df100_DDM["a_mean"] = [np.mean(df100_DDM["Boundary Separation"])]*len(df100_DDM.index)
    df100_DDM["a_sd"] = [np.std(df100_DDM["Boundary Separation"])]*len(df100_DDM.index)

    df100_DDM["ter_mean"] = [np.mean(df100_DDM["Non-decision Time"])]*len(df100_DDM.index)
    df100_DDM["ter_sd"] = [np.std(df100_DDM["Non-decision Time"])]*len(df100_DDM.index)

    # standardize
    df100_DDM["Drift Rate"] = (df100_DDM["Drift Rate"] - df100_DDM["v_mean"])/df100_DDM["v_sd"]
    df100_DDM["Boundary Separation"] = (df100_DDM["Boundary Separation"] - df100_DDM["a_mean"])/df100_DDM["a_sd"]
    df100_DDM["Non-decision Time"] = (df100_DDM["Non-decision Time"] - df100_DDM["ter_mean"])/df100_DDM["ter_sd"]
    
    if "Effective Inhibition" in list(df100_DDM.columns):
        df100_DDM["k_mean"] = [np.mean(df100_DDM["Effective Inhibition"])]*len(df100_DDM.index)
        df100_DDM["k_sd"] = [np.std(df100_DDM["Effective Inhibition"])]*len(df100_DDM.index)
        df100_DDM["Effective Inhibition"] = (df100_DDM["Effective Inhibition"] - df100_DDM["k_mean"])/df100_DDM["k_sd"]
    
    if model == "DDM":
        df_melted = pd.melt(df100_DDM, id_vars = ["Dataset", "Model"],
                            value_vars=['Drift Rate', 'Boundary Separation','Non-decision Time'],
                               var_name = "Parameter", value_name = "Difference")
        #print(1)
    if model == "OUM" or model == "DDMOUM":
        df_melted = pd.melt(df100_DDM, id_vars = ["Dataset", "Model"],
                            value_vars=['Drift Rate', 'Boundary Separation','Non-decision Time', "Effective Inhibition"],
                               var_name = "Parameter", value_name = "Difference")
        #print(2)
    
    return(df_melted)

In [5]:
def get_difference(dataset, df_SAT, model):
    if model == "DDM":
        S = np.array(df_SAT.loc[(df_SAT['SAT'] == "Speed")][["Drift Rate", "Boundary Separation", "Non-decision Time"]])
        A = np.array(df_SAT.loc[(df_SAT['SAT'] == "Accuracy")][["Drift Rate", "Boundary Separation", "Non-decision Time"]])
        dif = S-A
        df_dif = pd.DataFrame(dif,columns = ["Drift Rate", "Boundary Separation", "Non-decision Time"])
    elif model == "OUM" or model == "DDMOUM":
        S = np.array(df_SAT.loc[(df_SAT['SAT'] == "Speed")][["Drift Rate", "Boundary Separation", "Non-decision Time", "Effective Inhibition"]])
        A = np.array(df_SAT.loc[(df_SAT['SAT'] == "Accuracy")][["Drift Rate", "Boundary Separation", "Non-decision Time", "Effective Inhibition"]])
        dif = S-A
        df_dif = pd.DataFrame(dif,columns = ["Drift Rate", "Boundary Separation", "Non-decision Time", "Effective Inhibition"])
    else:
        print("Error: model not implemented")
    df_dif["Dataset"] = [dataset]*len(df_dif)
    df_dif["Model"] = [model]*len(df_dif)
    return(df_dif)

In [ ]:
def get_difference_DDM1(dataset, df_SAT, model):
    S = np.array(df_SAT.loc[(df_SAT['SAT'] == "Speed")][["Drift Rate", "Boundary Separation", "Non-decision Time"]])
    A = np.array(df_SAT.loc[(df_SAT['SAT'] == "Accuracy")][["Drift Rate", "Boundary Separation", "Non-decision Time"]])
    dif = S-A
    df_dif = pd.DataFrame(dif,columns = ["Drift Rate", "Boundary Separation", "Non-decision Time"])


    df_dif_melted = pd.melt(df_dif, value_vars=['Drift Rate', 'Boundary Separation','Non-decision Time'],
                           var_name = "Parameter", value_name = "Difference")

    df_dif_melted["Dataset"] = [dataset]*len(df_dif_melted)
    df_dif_melted["Model"] = [model]*len(df_dif_melted)
    return(df_dif_melted)

def get_difference_OUM1(dataset, df_SAT, model):
    S = np.array(df_SAT.loc[(df_SAT['SAT'] == "Speed")][["Drift Rate", "Boundary Separation", "Non-decision Time", "Effective Inhibition"]])
    A = np.array(df_SAT.loc[(df_SAT['SAT'] == "Accuracy")][["Drift Rate", "Boundary Separation", "Non-decision Time", "Effective Inhibition"]])
    dif = S-A
    df_dif = pd.DataFrame(dif,columns = ["Drift Rate", "Boundary Separation", "Non-decision Time", "Effective Inhibition"])
    df_dif_melted = pd.melt(df_dif, value_vars=['Drift Rate', 'Boundary Separation','Non-decision Time', "Effective Inhibition"],
                           var_name = "Parameter", value_name = "Difference")

    df_dif_melted["Dataset"] = [dataset]*len(df_dif_melted)
    df_dif_melted["Model"] = [model]*len(df_dif_melted)
    return(df_dif_melted)

In [ ]:
def plot_OUM(df_SAT_melted, comparisons_correction=None, figsize=(8,6)):

    box_pairs=[
       (("Drift Rate", "Speed"), ("Drift Rate", "Accuracy")), 
       (("Boundary Separation", "Speed"), ("Boundary Separation", "Accuracy")), 
       (("Non-decision Time", "Speed"), ("Non-decision Time", "Accuracy")),
       (("Effective Inhibition", "Speed"), ("Effective Inhibition", "Accuracy"))
      ]

    plt.figure(figsize=figsize)
    ax = sns.barplot(data=df_SAT_melted, x="Parameter", y="Value", hue="SAT")
    add_stat_annotation(ax, data=df_SAT_melted, x="Parameter", y="Value", hue="SAT",
                        box_pairs = box_pairs, text_format='star', loc='inside', verbose=2,
                        comparisons_correction=comparisons_correction, test='t-test_paired')

    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    plt.show()
    #plt.savefig('example_hue_layout.png', dpi=300, bbox_inches='tight')

In [ ]:
def plot_DDM(df_SAT_melted, comparisons_correction=None, figsize=(8,6)):

    box_pairs=[
       (("Drift Rate", "Speed"), ("Drift Rate", "Accuracy")), 
       (("Boundary Separation", "Speed"), ("Boundary Separation", "Accuracy")), 
       (("Non-decision Time", "Speed"), ("Non-decision Time", "Accuracy"))
      ]

    plt.figure(figsize=figsize)
    ax = sns.barplot(data=df_SAT_melted, x="Parameter", y="Value", hue="SAT")
    add_stat_annotation(ax, data=df_SAT_melted, x="Parameter", y="Value", hue="SAT",
                        box_pairs = box_pairs, text_format='star', loc='inside', verbose=2,
                        comparisons_correction=comparisons_correction, test='t-test_paired')

    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    plt.show()
    #plt.savefig('example_hue_layout.png', dpi=300, bbox_inches='tight')

In [ ]:
def DDM_OUM_QP_plot(dataset, data_dir,sample_size, 
                   correct, corrects,
                   cond, conds, rt,
                    ax=None):
    if ax is None:
        ax = plt.gca()
    
    quantiles = [.1, .3, .5, .7, .9]
    ## behavioral data
    RT_cor_s, RT_cor_a, RT_err_s, RT_err_a, nums = get_RT_quantile(data_dir = data_dir,sample_size= sample_size, 
                                                                   correct = correct, corrects = corrects,
                                                                   cond = cond, conds = conds, rt = rt)
    
    ax.plot([nums[0]/(nums[0]+nums[2])]*5, RT_cor_s,"o", color = 'black', fillstyle = 'none')
    ax.plot([nums[1]/(nums[1]+nums[3])]*5, RT_cor_a,"o", color = 'black', fillstyle = 'none')
    ax.plot([nums[2]/(nums[0]+nums[2])]*5, RT_err_s,"o", color = 'black', fillstyle = 'none')
    ax.plot([nums[3]/(nums[1]+nums[3])]*5, RT_err_a,"o", color = 'black', fillstyle = 'none')
    
    ## data generated from DDM
    params_DDM = pd.read_csv("DDM2v2a2terresult"+dataset+".csv")#.drop(labels=3, axis=0)
    RT_cor_s, RT_cor_a, RT_err_s, RT_err_a, nums = get_RT_quantile_DDM(params_all = params_DDM,sample_size= sample_size, 
                                                                   correct = correct, corrects = corrects,
                                                                   cond = cond, conds = conds, rt = rt)
    
    ax.plot([nums[0]/(nums[0]+nums[2])]*5, RT_cor_s,"x", color = '#008176', fillstyle = 'none')
    ax.plot([nums[1]/(nums[1]+nums[3])]*5, RT_cor_a,"x", color = '#008176', fillstyle = 'none')
    ax.plot([nums[2]/(nums[0]+nums[2])]*5, RT_err_s,"x", color = '#008176', fillstyle = 'none')
    ax.plot([nums[3]/(nums[1]+nums[3])]*5, RT_err_a,"x", color = '#008176', fillstyle = 'none')
    
    ## data generated from OUM
    params_OUM = pd.read_csv("OUM2v2k2terresult"+dataset+".csv")#.drop(labels=3, axis=0)
    RT_cor_s, RT_cor_a, RT_err_s, RT_err_a, nums = get_RT_quantile_OUM2v2k2ter(params_all = params_OUM,sample_size= sample_size, 
                                                                   correct = correct, corrects = corrects,
                                                                   cond = cond, conds = conds, rt = rt)
    
    ax.plot([nums[0]/(nums[0]+nums[2])]*5, RT_cor_s,"x", color = '#c1272d', fillstyle = 'none')
    ax.plot([nums[1]/(nums[1]+nums[3])]*5, RT_cor_a,"x", color = '#c1272d', fillstyle = 'none')
    ax.plot([nums[2]/(nums[0]+nums[2])]*5, RT_err_s,"x", color = '#c1272d', fillstyle = 'none')
    ax.plot([nums[3]/(nums[1]+nums[3])]*5, RT_err_a,"x", color = '#c1272d', fillstyle = 'none')
    
    # limits
    ax.set_xlim(0,1)
    ax.set_ylim(0,)
        
    ax.set_xlabel("Probability of Response")
    ax.set_ylabel("RT")
    #ax.legend(loc = 'upper right')
    return(ax)
#DDM_OUM_QP_plot(dataset)

In [ ]:
def DDM_OUM_09quantile_plot(dataset, data_dir,sample_size, correct, corrects,
                            cond, conds, rt,figsize=(8, 4),legend = False):
    #if ax is None:
    #    ax = plt.gca()
    
    figure, ax = plt.subplots(1,2,figsize=figsize)

    ## data generated from DDM
    params_DDM = pd.read_csv("DDM2v2a2terresult"+dataset+".csv")#.drop(labels=3, axis=0)
    RT_cor_s, RT_cor_a, RT_err_s, RT_err_a, nums = get_all_RT_quantile_DDM(params_all = params_DDM,sample_size= sample_size, 
                                                                   correct = correct, corrects = corrects,
                                                                   cond = cond, conds = conds, rt = rt)
    quantiles_DDM = pd.DataFrame({"cor_s":RT_cor_s[:,4],"cor_a":RT_cor_a[:,4], "err_s": RT_err_s[:,4],'err_a': RT_err_a[:,4]})

    ## data generated from OUM
    params_OUM = pd.read_csv("OUM2v2k2terresult"+dataset+".csv")#.drop(labels=3, axis=0)
    RT_cor_s, RT_cor_a, RT_err_s, RT_err_a, nums = get_all_RT_quantile_OUM2v2k2ter(params_all = params_OUM,sample_size= sample_size, 
                                                                   correct = correct, corrects = corrects,
                                                                   cond = cond, conds = conds, rt = rt)
    quantiles_OUM = pd.DataFrame({"cor_s":RT_cor_s[:,4],"cor_a":RT_cor_a[:,4], "err_s": RT_err_s[:,4],'err_a': RT_err_a[:,4]})
    
    ran = np.arange(0,max(quantiles_DDM.max(axis=None),quantiles_OUM.max(axis=None))+0.5,0.1)


    ax[0].scatter(quantiles_DDM["cor_s"],quantiles_OUM["cor_s"], c = [[1,1,1]], edgecolors = "#1F77B4", alpha = 0.7, label = "Speed")
    ax[0].scatter(quantiles_DDM["cor_a"],quantiles_OUM["cor_a"], c = [[1,1,1]], edgecolors = "#FF7F0E", alpha = 0.7, label = "Accuracy")
    ax[0].set_xlabel("0.9 Quantile of DDM")
    ax[0].set_ylabel("0.9 Quantile of OUM")
    ax[0].plot(ran,ran,c = "black")
    ax[0].set_title("Correct Choices")

    ax[1].scatter(quantiles_DDM["err_s"],quantiles_OUM["err_s"], c = [[1,1,1]], edgecolors = "#1F77B4", alpha = 0.7, label = "Speed")
    ax[1].scatter(quantiles_DDM["err_a"],quantiles_OUM["err_a"], c = [[1,1,1]], edgecolors = "#FF7F0E", alpha = 0.7, label = "Accuracy")
    ax[1].set_xlabel("0.9 Quantile of DDM")
    ax[1].set_title("Incorrect Choices")
    
    
    ax[1].plot(ran,ran,c = "black")

    
    #ax[1].set_ylabel("0.9 Quantile of OUM")
    #ax[1].set_yticklabels([])
    if legend:
        ax[1].legend(loc='upper right', bbox_to_anchor=(1.5, 1))
    
    return ax


In [ ]:
def get_all_RT_quantile_DDM(params_all, sample_size, correct, corrects, cond, conds, rt, quantiles = [.1, .3, .5, .7, .9]):

    RT_cor_ss, RT_cor_as, RT_err_ss, RT_err_as, nums = np.zeros((sample_size,5)),np.zeros((sample_size,5)),np.zeros((sample_size,5)),np.zeros((sample_size,5)),np.zeros((sample_size,4))
    for i in range(sample_size):
        #print(i)
        params = np.array(params_all)[i,1:7]
        
        
        RT_cor_s, RT_cor_a, RT_err_s, RT_err_a, num = get_RT_quantile_DDM_sub(params = params)
        #print(RT_err_a)
        RT_cor_ss[i,:] = RT_cor_s
        RT_cor_as[i,:] = RT_cor_a
        RT_err_ss[i,:] = RT_err_s
        RT_err_as[i,:] = RT_err_a
        nums[i,:] = num
    
    return RT_cor_ss, RT_cor_as, RT_err_ss, RT_err_as, nums


In [ ]:
def get_all_RT_quantile_OUM2v2k2ter(params_all, sample_size, correct, corrects, cond, conds, rt, quantiles = [.1, .3, .5, .7, .9]):

    RT_cor_ss, RT_cor_as, RT_err_ss, RT_err_as, nums = np.zeros((sample_size,5)),np.zeros((sample_size,5)),np.zeros((sample_size,5)),np.zeros((sample_size,5)),np.zeros((sample_size,4))
    for i in range(sample_size):

        params = np.array(params_all)[i,1:8]
        RT_cor_s, RT_cor_a, RT_err_s, RT_err_a, num = get_RT_quantile_OUM2v2k2ter_sub(params = params)
        #print(RT_err_a)
        RT_cor_ss[i,:] = RT_cor_s
        RT_cor_as[i,:] = RT_cor_a
        RT_err_ss[i,:] = RT_err_s
        RT_err_as[i,:] = RT_err_a
        nums[i,:] = num
    
    return RT_cor_ss, RT_cor_as, RT_err_ss, RT_err_as, nums


In [ ]:
def get_all_RT_quantile(data_dir, sample_size, correct, corrects, cond, conds, rt, quantiles = [.1, .3, .5, .7, .9]):

    RT_cor_ss, RT_cor_as, RT_err_ss, RT_err_as, nums = np.zeros((sample_size,5)),np.zeros((sample_size,5)),np.zeros((sample_size,5)),np.zeros((sample_size,5)),np.zeros((sample_size,4))
    for i in range(sample_size):
        #print(i)
        path = os.path.join(data_dir+str(i+1)+".csv")
        df = pd.read_csv(path, sep = ",")
        #print(df)
        df = df[df[rt]<5]
        RT_cor_s, RT_cor_a, RT_err_s, RT_err_a, num = get_RT_quantile_sub(
            df = df, rt = rt,
            correct = correct, corrects = corrects,
            cond = cond, conds = conds)
        #print(RT_cor_s)
        RT_cor_ss[i,:] = RT_cor_s
        RT_cor_as[i,:] = RT_cor_a
        RT_err_ss[i,:] = RT_err_s
        RT_err_as[i,:] = RT_err_a
        nums[i,:] = num
        
        #print(RT_err_s)

    
    return RT_cor_ss, RT_cor_as, RT_err_ss, RT_err_as, nums


In [3]:
def get_ks_data(data_dir, sample_size, correct, corrects, cond, conds, rt):

    ks_cor_s, ks_cor_a, ks_err_s, ks_err_a = [],[],[],[]
    for i in range(sample_size):
        #print(i)
        path = os.path.join(data_dir+str(i+1)+".csv")
        df = pd.read_csv(path, sep = ",")
        #print(df)
        df = df[df[rt]<5]
        
        df_s = df[df[cond] == conds[0]]
        df_cor_s = df_s[df_s[correct]==corrects[0]]
        df_err_s = df_s[df_s[correct]==corrects[1]]

        df_a = df[df[cond] == conds[1]]
        df_cor_a = df_a[df_a[correct]==corrects[0]]
        df_err_a = df_a[df_a[correct]==corrects[1]]

        
        ks_cor_s.append(get_ks_stat(df_cor_s, eval_model = False, rt = rt))
        ks_err_s.append(get_ks_stat(df_err_s, eval_model = False, rt = rt))
        ks_cor_a.append(get_ks_stat(df_cor_a, eval_model = False, rt = rt))
        ks_err_a.append(get_ks_stat(df_err_a, eval_model = False, rt = rt))
        
        ks_data = pd.DataFrame({"cor_s":ks_cor_s,
                                "cor_a":ks_cor_a,
                                "err_s": ks_err_s,
                                'err_a': ks_err_a})
    
    return ks_data


In [ ]:
def get_ks_stat(df, eval_model = True, rt = "RT"):
    #print(len(df["RT"]))
    if eval_model:
        if len(df[rt])<1000:
            return 0
    else:
        if len(df[rt])<10:
            return 0
    
    normal_samp = np.random.normal(loc=np.mean(df[rt]), scale=np.std(df[rt]), size=len(df[rt]))
    ks = scipy.stats.kstest(df[0:1000][rt], normal_samp)[0]
    return ks


def get_ks_DDM(params_all, sample_size, cond, conds, T_dur=5):

    ks_all = pd.DataFrame()

    for i in range(sample_size):
        params = np.array(params_all)[i,1:7]
        
        Vspeed,Vacc, Bspeed,Bacc,tSpeed,tAcc = params[0],params[1],params[2],params[3],params[4],params[5]

        model = Model(name='Simple model (fitted)',
                          drift=DriftConstantSpeedAcc(Vspeed=Vspeed, Vacc=Vacc), 
                          noise=NoiseConstant(noise=1),
                          bound=BoundSpeedAcc(Bspeed=Bspeed,Bacc=Bacc),
                          overlay=NondecisionSpeedAcc(tSpeed=tSpeed,tAcc=tAcc),
                          dx=.001, dt=.001, T_dur=T_dur)
        sol_s = model.solve({cond:conds[0]})
        samp_s = sol_s.resample(100000)
        data_s = samp_s.to_pandas_dataframe(drop_undecided = True)

        sol_a = model.solve({cond:conds[1]})
        samp_a = sol_a.resample(100000)
        data_a = samp_a.to_pandas_dataframe(drop_undecided = True)


        ks_cor_s = get_ks_stat(data_s[data_s['choice']==1])
        ks_err_s = get_ks_stat(data_s[data_s['choice']==0])
        ks_cor_a = get_ks_stat(data_a[data_a['choice']==1])
        ks_err_a = get_ks_stat(data_a[data_a['choice']==0])

        ks = pd.DataFrame({"cor_s":[ks_cor_s], "err_s": [ks_err_s], "cor_a":[ks_cor_a],'err_a': [ks_err_a]})
        ks["Participant"] = i

        ks_all = pd.concat((ks_all,ks))
        
    return ks_all

def get_ks_OUM(params_all, sample_size, cond, conds, T_dur=5):

    ks_all = pd.DataFrame()

    for i in range(sample_size):
        params = np.array(params_all)[i,1:8]
        
        Vspeed,Vacc, Kspeed,Kacc,B,tSpeed,tAcc = params[0],params[1],params[2],params[3],params[4],params[5],params[6]

        model = Model(name='Simple model (fitted)',
                          drift=DriftBothSpeedAcc(Vspeed=Vspeed, Vacc=Vacc, Kspeed=Kspeed, Kacc=Kacc),                              
                          noise=NoiseConstant(noise=1),
                          bound=BoundConstant(B=B),
                          overlay=NondecisionSpeedAcc(tSpeed=tSpeed,tAcc=tAcc),
                          dx=.001, dt=.001, T_dur=T_dur)
        sol_s = model.solve({cond:conds[0]})
        samp_s = sol_s.resample(100000)
        data_s = samp_s.to_pandas_dataframe(drop_undecided = True)

        sol_a = model.solve({cond:conds[1]})
        samp_a = sol_a.resample(100000)
        data_a = samp_a.to_pandas_dataframe(drop_undecided = True)


        ks_cor_s = get_ks_stat(data_s[data_s['choice']==1])
        ks_err_s = get_ks_stat(data_s[data_s['choice']==0])
        ks_cor_a = get_ks_stat(data_a[data_a['choice']==1])
        ks_err_a = get_ks_stat(data_a[data_a['choice']==0])

        ks = pd.DataFrame({"cor_s":[ks_cor_s], "err_s": [ks_err_s], "cor_a":[ks_cor_a],'err_a': [ks_err_a]})
        ks["Participant"] = i

        ks_all = pd.concat((ks_all,ks))
        
    return ks_all

In [ ]:
def get_ks_DDM_matched(params_all, sample_size, cond, conds, RT = "Time", T_dur=5):

    ks_all = pd.DataFrame()

    for i in range(sample_size):
        params = np.array(params_all)[i,1:7]
        
        Vspeed,Vacc, Bspeed,Bacc,tSpeed,tAcc = params[0],params[1],params[2],params[3],params[4],params[5]

        model = Model(name='Simple model (fitted)',
                          drift=DriftConstantSpeedAcc(Vspeed=Vspeed, Vacc=Vacc), 
                          noise=NoiseConstant(noise=1),
                          bound=BoundSpeedAcc(Bspeed=Bspeed,Bacc=Bacc),
                          overlay=NondecisionSpeedAcc(tSpeed=tSpeed,tAcc=tAcc),
                          dx=.001, dt=.001, T_dur=T_dur)
        sol_s = model.solve({cond:conds[0]})
        samp_s = sol_s.resample(100000)
        data_s = samp_s.to_pandas_dataframe(drop_undecided = True)

        sol_a = model.solve({cond:conds[1]})
        samp_a = sol_a.resample(100000)
        data_a = samp_a.to_pandas_dataframe(drop_undecided = True)
        
        path = os.path.join(data_dir+str(i+1)+".csv")
        df = pd.read_csv(path, sep = ",")
        #print(df)
        df = df[df[RT]<5]
        
        df_s = df[df[cond] == conds[0]]
        n_cor_s = len(df_s[df_s[correct]==corrects[0]])
        n_err_s = len(df_s[df_s[correct]==corrects[1]])

        df_a = df[df[cond] == conds[1]]
        n_cor_a = len(df_a[df_a[correct]==corrects[0]])
        n_err_a = len(df_a[df_a[correct]==corrects[1]])

        #print(data_s[data_s['choice']==1][0:n_cor_s])
        ks_cor_s = get_ks_stat(data_s[data_s['choice']==1][0:n_cor_s],eval_model = False)
        ks_err_s = get_ks_stat(data_s[data_s['choice']==0][0:n_err_s],eval_model = False)
        ks_cor_a = get_ks_stat(data_a[data_a['choice']==1][0:n_cor_a],eval_model = False)
        ks_err_a = get_ks_stat(data_a[data_a['choice']==0][0:n_err_a],eval_model = False)

        ks = pd.DataFrame({"cor_s":[ks_cor_s], "err_s": [ks_err_s], "cor_a":[ks_cor_a],'err_a': [ks_err_a]})
        ks["Participant"] = i

        ks_all = pd.concat((ks_all,ks))
        
    return ks_all

In [ ]:
def get_ks_OUM_matched(params_all, sample_size, cond, conds, RT = "Time", T_dur=5):

    ks_all = pd.DataFrame()

    for i in range(sample_size):
        params = np.array(params_all)[i,1:8]
        
        Vspeed,Vacc, Kspeed,Kacc,B,tSpeed,tAcc = params[0],params[1],params[2],params[3],params[4],params[5],params[6]

        model = Model(name='Simple model (fitted)',
                          drift=DriftBothSpeedAcc(Vspeed=Vspeed, Vacc=Vacc, Kspeed=Kspeed, Kacc=Kacc),                              
                          noise=NoiseConstant(noise=1),
                          bound=BoundConstant(B=B),
                          overlay=NondecisionSpeedAcc(tSpeed=tSpeed,tAcc=tAcc),
                          dx=.001, dt=.001, T_dur=T_dur)
        sol_s = model.solve({cond:conds[0]})
        samp_s = sol_s.resample(100000)
        data_s = samp_s.to_pandas_dataframe(drop_undecided = True)

        sol_a = model.solve({cond:conds[1]})
        samp_a = sol_a.resample(100000)
        data_a = samp_a.to_pandas_dataframe(drop_undecided = True)
        
        path = os.path.join(data_dir+str(i+1)+".csv")
        df = pd.read_csv(path, sep = ",")
        #print(df)
        df = df[df[RT]<5]
        
        df_s = df[df[cond] == conds[0]]
        n_cor_s = len(df_s[df_s[correct]==corrects[0]])
        n_err_s = len(df_s[df_s[correct]==corrects[1]])

        df_a = df[df[cond] == conds[1]]
        n_cor_a = len(df_a[df_a[correct]==corrects[0]])
        n_err_a = len(df_a[df_a[correct]==corrects[1]])

        #print(data_s[data_s['choice']==1][0:n_cor_s])
        ks_cor_s = get_ks_stat(data_s[data_s['choice']==1][0:n_cor_s],eval_model = False)
        ks_err_s = get_ks_stat(data_s[data_s['choice']==0][0:n_err_s],eval_model = False)
        ks_cor_a = get_ks_stat(data_a[data_a['choice']==1][0:n_cor_a],eval_model = False)
        ks_err_a = get_ks_stat(data_a[data_a['choice']==0][0:n_err_a],eval_model = False)

        ks = pd.DataFrame({"cor_s":[ks_cor_s], "err_s": [ks_err_s], "cor_a":[ks_cor_a],'err_a': [ks_err_a]})
        ks["Participant"] = i

        ks_all = pd.concat((ks_all,ks))
        
    return ks_all

In [ ]:
def plot_normality_correct(ks_DDM,ks_OUM, ax=None):
    if ax is None:
        ax = plt.gca()
    
    ax.scatter(ks_DDM["cor_s"], ks_OUM["cor_s"],c = [[1,1,1]], edgecolors = "#1F77B4", alpha = 0.7, label = "Speed")
    ran = np.arange(0,max(ks_DDM["cor_s"].max(axis=None),ks_OUM["cor_s"].max(axis=None))+0.02,0.0001)
    ax.plot(ran,ran,c = "black")
    
    ax.scatter(ks_DDM["cor_a"], ks_OUM["cor_a"],c = [[1,1,1]], edgecolors = "#FF7F0E", alpha = 0.7, label = "Speed")
    ran = np.arange(0,max(ks_DDM["cor_a"].max(axis=None),ks_OUM["cor_a"].max(axis=None))+0.02,0.0001)
    ax.plot(ran,ran,c = "black")
    ax.set_title("Correct Choices")
    ax.set_xlabel("K-S Statistic of DDM")
    ax.set_ylabel("K-S Statistic of OUM")
    
    return ax

def plot_normality_error(ks_DDM,ks_OUM, ax=None):
    if ax is None:
        ax = plt.gca()
    
    ax.scatter(ks_DDM["err_s"], ks_OUM["err_s"],c = [[1,1,1]], edgecolors = "#1F77B4", alpha = 0.7, label = "Speed")
    ran = np.arange(0,max(ks_DDM["err_s"].max(axis=None),ks_OUM["err_s"].max(axis=None))+0.02,0.0001)
    ax.plot(ran,ran,c = "black")
    
    ax.scatter(ks_DDM["err_a"], ks_OUM["err_a"],c = [[1,1,1]], edgecolors = "#FF7F0E", alpha = 0.7, label = "Speed")
    ran = np.arange(0,max(ks_DDM["err_a"].max(axis=None),ks_OUM["err_a"].max(axis=None))+0.02,0.0001)
    ax.plot(ran,ran,c = "black")
    ax.set_title("Incorrect Choices")
    ax.set_xlabel("K-S Statistic of DDM")
    #ax.set_ylabel("K-S Statistic of OUM")
    
    return ax